## **Count Vectorizer를 사용한 감성분석**

In [1]:
!pip install scikit-learn

In [2]:
import pandas as pd
# csv 형식의 training 데이터를 로드한다.
data=pd.read_csv('drive/MyDrive/Colab Notebooks/dataset/train.csv')
data.head()

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0


### 결측치 확인

In [3]:
def check_missing(dataframe):
  missing_col=[]
  for col in dataframe.columns:
    missing_values=sum(dataframe[col].isna())
    is_missing=True if missing_values>=1 else False
    if is_missing:
      print(f'결측치가 있는 컬럼은:{col}입니다.')
      print(f'해당 컬럼에 총 {missing_values}개의 결측치가 존재합니다.')
  if missing_col==[]:
    print('결측치가 존재하지 않습니다.')
  return missing_col

missing_col=check_missing(data)

결측치가 존재하지 않습니다.


In [33]:
X=data.document # training 데이터에서 문서 추출
y=data.label # training 데이터에서 라벨 추출

In [34]:
X.head()

0                  영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐
1               히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...
2       괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~
3                아무래도 20년도지난작품이라 지금보기는너무유치하다
4    지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.
Name: document, dtype: object

In [35]:
y.head()

0    0
1    1
2    1
3    0
4    0
Name: label, dtype: int64

## **Count Vectorizer의 이해**

#### 컴퓨터는 문장(텍스트)를 계산할 수 없다.
#### 컴퓨터가 계산을 하기 위해서는 먼저 문장을 숫자로 바꾼 다음 계산하도록 해야 한다.
#### 이것을 워드 임베딩(word embedding)이라고 한다.
#### CountVectorizer 기법을 사용해 문장을 숫자로 표현하고자 한다.

#### CountVectorizer는 입력된 문장을 토큰화(Tokenize)하여 토큰의 등장 빈도 벡터로 바꿔주는 기법이다
####(* 토큰화란 문장을 의미가 있는 최소의 단위로 쪼개는 것을 의미한다.)
#### 예를 들어 "hello, I am a data scientist!" 라는 문장을 띄어쓰기 기준으로 토큰화 시키면 8개의 토큰으로 토큰화 된다.

#### 토큰화하는 방식은 여러가지가 있지만 이번 프로젝트에서는 띄어쓰기를 기준으로 토큰화를 진행한다.

In [36]:
from sklearn.feature_extraction.text import CountVectorizer # sklearn 패키지의 CountVectorizer import
sample_vectorizer=CountVectorizer() # 객체 생성

#### CountVectorizer를 학습시킨다.
#### 어떤 단어들을 사용할 지, 어떤 단어가 중요도를 가지는지 학습하는 과정이다.
#### CounterVectorizer의 경우 단어의 출현 빈도를 기준으로 문장을 벡터(=숫자의 집합)으로 바꿔준다

In [37]:
sample_text1=['hello, my name is chaeyeon and I am a data scientist!']
sample_vectorizer.fit(sample_text1) # CountVertorize 학습

CountVectorizer()

#### sample_vectorizer 가 학습한 단어 목록을 Vocabulary라고 하며, 앞으로 sample_vectorizer는 이 Vocab을 기준으로 새로운 문장을 숫자로 바꿔줄 것이다.

In [38]:
print(sample_vectorizer.vocabulary_) # Vocabulary

{'hello': 4, 'my': 6, 'name': 7, 'is': 5, 'chaeyeon': 2, 'and': 1, 'am': 0, 'data': 3, 'scientist': 8}


In [39]:
from IPython.display import Image
Image("./image.png")

#### sklearn의 CountVectorizer는 "I","a"와 같이 길이가 2 이하인 단어들과, ",","!" 같은 특수 문자를 의미가 적다고 판단하여 제외시킨다.

#### 그래서 최종적으로 9개의 단어들이 Vocab에 포함된다

In [40]:
sample_text2=["you are learning dacon data science"]

sample_vector=sample_vectorizer.transform(sample_text2)
print(sample_vector.toarray())

[[0 0 0 1 0 0 0 0 0]]


#### "you are learning dacon data science" 문장을 sample_vectorizer를 활용해 transform 했다.
#### 단어들의 출현 빈도로 이루어진 크기 9인 벡터가 출력됐다. 이 벡터를 BoW라고 한다.
#### BoW란 "Bag of Words"의 약자로 CountVectorizer로 변환된 단어의 집합을 말한다.
#### sample_text2 문장에는 "data"가 1번 출현했다.
#### 따라서 "data"에 해당하는 인덱스의 값이 1로 표시되어 있고, 등장하지 않은 단어들은 0으로 표시된다.

In [41]:
sample_text3=['you are learning dacon data science with movie data']

sample_vector2=sample_vectorizer.transform(sample_text3)
print(sample_vector2.toarray())

[[0 0 0 2 0 0 0 0 0]]


#### CounterVectorizer 를 하나의 문장을 사용해 학습 시켰기에 생성된 Vocab과 BoW의 크기는 모두 9였다.
#### 하지만 더 큰 데이터를 사용해 CountVectorizer를 학습시킨다면 Vocab과 BoW의 크기는 더욱 증가할 것이다.

## **영화 리뷰 데이터**

#### 새로운 객체를 생성하여 CountVectorizer를 할당해준다.
#### 다음 미리 준비해두었던 데이터를 사용해 CountVectorizer를 학습시켜준다.

In [42]:
vectorizer=CountVectorizer() # countvectorizer 생성
vectorizer.fit(X) # countvectorizer 학습
X=vectorizer.transform(X)

#### CountVectorizer를 학습 시켰다.
#### X는 이제 자연어가 아닌 숫자, 벡터 데이터로 이루어져 있다.
#### 이제는 머신러닝 모델의 input 값으로 적절한 데이터의 형식을 갖춘 것이다!

## Logistic Regression

#### 분류 문제에 많이 사용 되는 로지스틱 회귀모델 사용

In [44]:
from sklearn.linear_model import LogisticRegression # 모델 불러오기
model=LogisticRegression() # 객체에 모델 할당
model.fit(X,y) # 모델 학습

LogisticRegression()

In [45]:
# run model
X_pred=vectorizer.transform(['영화 꿀잼'])
y_pred=model.predict(X_pred)
print(y_pred)
# 0:negative, 1:positive

[1]


## test 데이터를 예측해 제출 파일 생성

In [47]:
test=pd.read_csv("drive/MyDrive/Colab Notebooks/dataset/test.csv") # 파일 읽기
test.head() # 파일 확인

,id,document
0,1,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!
2,3,Childhood fantasy
3,4,멋있는 영화입니다. 잊을 수 없는!
4,5,너무 감동적이네요 펑펑 울었습니다


#### test 데이터를 transform 한 뒤, test 데이터의 label을 예측

In [48]:
test_X=test.document # 문서 데이터 생성
test_X_vect=vectorizer.transform(test_X) # 문서 데이터 transform
# test 데이터를 대상으로 fit_transform 메소드를 실행하는 것은 test 데이터를 활용해 vectorizer를 학습 시키는 것으로 data leakage에 해당한다.
# 따라서 test 데이터로 모델을 학습시키지 않는다. train데이터로 학습된 모델을 사용.

pred=model.predict(test_X_vect) # test 데이터 예측
print(pred)

[0 1 0 ... 1 0 0]


In [49]:
submission=pd.read_csv('drive/MyDrive/Colab Notebooks/dataset/submission.csv')
submission.head() # 제출 파일이 잘 생성되었는지 확인

,id,label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [52]:
submission['label']=pred # 예측 값 넣어주기
submission.head(10) # 데이터가 잘 들어갔는지 확인

,id,label
0,1,0
1,2,1
2,3,0
3,4,1
4,5,1
5,6,0
6,7,0
7,8,0
8,9,1
9,10,1


In [51]:
# submission을 csv 파일로 저장
# index=False => 추가적인 id를 부여할 필요가 없다는 뜻이다.
submission.to_csv('drive/MyDrive/Colab Notebooks/dataset/submission.csv',index=False)